In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly
import cufflinks as cf
from lifelines import KaplanMeierFitter

import matplotlib.pyplot as plt

import scipy.stats as stats
from statsmodels.stats.weightstats import ztest

from lifelines.statistics import logrank_test, multivariate_logrank_test

cf.go_offline()

pd.options.plotting.backend = 'plotly'

In [2]:
data = pd.read_csv('../data/sinac_2020.csv', low_memory=False)

In [3]:
data

,NACIOEXTRANJERO,ENTIDADNACIMIENTO,MUNICIPIONACIMIENTO,EDAD,SECONSIDERAINDIGENA,HABLALENGUAINDIGENA,FECHANACIMIENTOMADRE,ESTADOCONYUGAL,RESIDEEXTRANJERO,ENTIDADRESIDENCIA,...,TIPOMEDICOATENDIO,ENTIDADFEDERATIVAPARTO,MUNICIPIOPARTO,LOCALIDADPARTO,CERTIFICADOPOR,CLUESCERTIFICA,ENTIDADFEDERATIVACERTIFICA,MUNICIPIOCERTIFICA,LOCALIDADCERTIFICA,FECHACERTIFICADO
0,2,14,91,29,2,2,25/11/1990,5,2,14,...,11.0,14,91,1,7,NaN,14,91,1,09/01/2020
1,2,27,2,26,2,2,26/03/1993,5,2,27,...,NaN,27,5,74,5,NaN,27,5,1,07/01/2020
2,2,15,104,25,2,2,20/05/1994,4,2,15,...,11.0,15,33,1,2,NaN,15,33,1,11/01/2020
3,2,7,101,22,2,2,26/05/1997,4,2,7,...,11.0,7,101,1,1,NaN,7,101,1,15/01/2020
4,2,20,533,22,2,2,25/09/1997,4,2,20,...,11.0,20,59,1,4,NaN,20,59,1,10/01/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747842,1,88,999,34,2,2,10/02/1985,1,2,14,...,11.0,14,67,1,1,9997,14,67,1,03/01/2020
1747843,1,88,999,30,2,2,07/07/1985,5,2,14,...,11.0,14,39,1,1,9997,14,39,1,08/01/2020
1747844,2,16,108,27,2,2,20/02/1992,5,2,14,...,11.0,14,67,1,1,9997,14,67,1,08/01/2020
1747845,2,16,98,33,2,2,13/10/1986,4,2,14,...,11.0,14,98,1,1,9997,14,98,1,11/01/2020


In [4]:
data.columns

Index(['NACIOEXTRANJERO', 'ENTIDADNACIMIENTO', 'MUNICIPIONACIMIENTO', 'EDAD',
       'SECONSIDERAINDIGENA', 'HABLALENGUAINDIGENA', 'FECHANACIMIENTOMADRE',
       'ESTADOCONYUGAL', 'RESIDEEXTRANJERO', 'ENTIDADRESIDENCIA',
       'MUNICIPIORESIDENCIA', 'LOCALIDADRESIDENCIA', 'NUMEROEMBARAZOS',
       'HIJOSNACIDOSMUERTOS', 'HIJOSNACIDOSVIVOS', 'HIJOSSOBREVIVIENTES',
       'CONDICIONHIJOANTERIOR', 'VIVEHIJOANTERIOR', 'ORDENNACIMIENTO',
       'ATENCIONPRENATAL', 'TRIMESTREPRIMERCONSULTA', 'TOTALCONSULTAS',
       'SOBREVIVIOPARTO', 'AFILIACION', 'ESCOLARIDAD', 'INTERRUMPIOESTUDIOS',
       'CLAVEOCUPACIONHABITUAL', 'TRABAJAACTUALMENTE', 'EDADPADRE',
       'FECHANACIMIENTO', 'HORANACIMIENTO', 'SEXO', 'EDADGESTACIONAL', 'TALLA',
       'PESO', 'APGAR', 'SILVERMAN', 'TAMIZAUDITIVO', 'VACUNA_BCG',
       'VACUNAHEPATITIS_B', 'VITAMINA_A', 'VITAMINA_K', 'PRODUCTOEMBARAZO',
       'ORDENPRODUCTO', 'TOTALPRODUCTOS', 'CODIGOCIEANOMALIA1',
       'CODIGOCIEANOMALIA2', 'LUGARNACIMIENTO', 'CLUES',

### Análisis de supervivencia

In [5]:
def survival(data: pd.DataFrame, group_field: str,
             time_field: str, event_field: str) -> tuple:
  """
  Función que recibe un DF y filtra por columna a partir del campo de 
  agrupación. 

  Regresa
    Plot de Función de Supervivencia
    Test de Logrank
  """

  model = KaplanMeierFitter()
  results = []
  con_exp = []

  for i in data[group_field].unique():
    group = data[data[group_field] == i]
    T = group[time_field]
    E = group[event_field]
    con_exp.append([T, E])
    model.fit(T, E, label=str(i))
    results.append(model.survival_function_)

  survival = pd.concat(results, axis=1)

  log_result = logrank_test(con_exp[0][0], con_exp[1][0], 
                            event_observed_A=con_exp[0][1], 
                            event_observed_B=con_exp[1][1])
  
  return survival, log_result

In [6]:
data_indigena = data[((data['SECONSIDERAINDIGENA'] == 1) | (data['SECONSIDERAINDIGENA'] == 2)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_indigena['SECONSIDERAINDIGENA'].replace(2, 'No', inplace=True)
data_indigena['SECONSIDERAINDIGENA'].replace(1, 'Si', inplace=True)

In [7]:
rates, logrank = survival(data_indigena, 'SECONSIDERAINDIGENA', 'EDAD', 'SOBREVIVIOPARTO')
print(logrank)
rates.plot(kind='line',
           title='Tasa de Mortalidad Materna por orígen étnico')

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
         112.73 <0.005     85.07


In [8]:
data['NUMEROEMBARAZOS'].unique()

array([ 2,  6,  1,  3,  5,  4, 10,  7,  9,  8, 11, 99, 12, 13, 16, 15, 20,
       14, 19, 25, 21], dtype=int64)

In [9]:
data_num_emb = data[(data['NUMEROEMBARAZOS'] != 99) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]

In [10]:
rates, logrank = survival(data_num_emb, 'NUMEROEMBARAZOS', 'EDAD', 'SOBREVIVIOPARTO')
print(logrank)
rates.plot(kind='line',
           title='Tasa de Mortalidad Materna por cantidad de embarazos previos')

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
       20816.92 <0.005       inf


In [11]:
data_prenatal = data[((data['ATENCIONPRENATAL'] == 1) | (data['ATENCIONPRENATAL'] == 2)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_prenatal['ATENCIONPRENATAL'].replace(2, 'No', inplace=True)
data_prenatal['ATENCIONPRENATAL'].replace(1, 'Si', inplace=True)

In [12]:
rates, logrank = survival(data_prenatal, 'ATENCIONPRENATAL', 'EDAD', 'SOBREVIVIOPARTO')
print(logrank)
rates.plot(kind='line',
           title='Tasa de Mortalidad Materna por cuidados prenatales')

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
        2766.83 <0.005       inf


In [13]:
data['TRIMESTREPRIMERCONSULTA'].unique()

array([1, 3, 2, 0, 9, 8], dtype=int64)

In [14]:
data_prim_consulta = data[((data['TRIMESTREPRIMERCONSULTA'] != 8) & (data['TRIMESTREPRIMERCONSULTA'] != 9)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_prim_consulta['TRIMESTREPRIMERCONSULTA'].replace(0, 'No tuvo', inplace=True)

In [15]:
rates, logrank = survival(data_prim_consulta, 'TRIMESTREPRIMERCONSULTA', 'EDAD', 'SOBREVIVIOPARTO')
print(logrank)
rates.plot(kind='line',
           title='Tasa de Mortalidad Materna por trimestre de primer cita')

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
        3271.34 <0.005       inf


In [16]:
data['TRABAJAACTUALMENTE'].unique()

array([1, 2, 8, 0, 9], dtype=int64)

In [17]:
data_trabajo = data[((data['TRABAJAACTUALMENTE'] == 1) | (data['TRABAJAACTUALMENTE'] == 2)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_trabajo['TRABAJAACTUALMENTE'].replace(2, 'No trabaja', inplace=True)
data_trabajo['TRABAJAACTUALMENTE'].replace(1, 'Trabaja', inplace=True)

In [18]:
rates, logrank = survival(data_trabajo, 'TRABAJAACTUALMENTE', 'EDAD', 'SOBREVIVIOPARTO')
rates.dropna().plot(kind='line',
           title='Tasa de Mortalidad Materna por estatus laboral')

In [19]:
data_entidad = data[((data['ENTIDADNACIMIENTO'] != 88) & (data['ENTIDADNACIMIENTO'] != 99) & (data['ENTIDADNACIMIENTO'] != 0)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_entidad['OrganizacionTerritorial'] = np.where(data_entidad['ENTIDADNACIMIENTO'] == 9, 'Capital', 'Interior')

In [20]:
rates, logrank = survival(data_entidad, 'OrganizacionTerritorial', 'EDAD', 'SOBREVIVIOPARTO')
rates.dropna().plot(kind='line',
           title='Tasa de Mortalidad Materna por lugar de nacimiento')

In [21]:
estados = {1:'AGUASCALIENTES', 2:'BAJA CALIFORNIA', 3:'BAJA CALIFORNIA SUR', 4:'CAMPECHE',
           5:'COAHUILA DE ZARAGOZA', 6:'COLIMA', 7:'CHIAPAS', 8:'CHIHUAHUA', 9:'CIUDAD DE MÉXICO',
           10:'DURANGO', 11:'GUANAJUATO', 12:'GUERRERO', 13:'HIDALGO', 14:'JALISCO', 15:'MÉXICO',
           16:'MICHOACÁN DE OCAMPO', 17:'MORELOS', 18:'NAYARIT', 19:'NUEVO LEÓN', 20:'OAXACA',
           21:'PUEBLA', 22:'QUERÉTARO', 23:'QUINTANA ROO', 24:'SAN LUIS POTOSÍ', 25:'SINALOA',
           26:'SONORA', 27:'TABASCO', 28:'TAMAULIPAS', 29:'TLAXCALA',
           30:'VERACRUZ DE IGNACIO DE LA LLAVE', 31:'YUCATÁN', 32:'ZACATECAS'}

data_entidad = data_entidad.replace({'ENTIDADNACIMIENTO': estados})

rates, logrank = survival(data_entidad, 'ENTIDADNACIMIENTO', 'EDAD', 'SOBREVIVIOPARTO')
rates.dropna().plot(kind='line',
           title='Tasa de Mortalidad Materna por lugar de nacimiento')

In [22]:
estados = {1:'AGUASCALIENTES', 2:'BAJA CALIFORNIA', 3:'BAJA CALIFORNIA SUR', 4:'CAMPECHE',
           5:'COAHUILA DE ZARAGOZA', 6:'COLIMA', 7:'CHIAPAS', 8:'CHIHUAHUA', 9:'CIUDAD DE MÉXICO',
           10:'DURANGO', 11:'GUANAJUATO', 12:'GUERRERO', 13:'HIDALGO', 14:'JALISCO', 15:'MÉXICO',
           16:'MICHOACÁN DE OCAMPO', 17:'MORELOS', 18:'NAYARIT', 19:'NUEVO LEÓN', 20:'OAXACA',
           21:'PUEBLA', 22:'QUERÉTARO', 23:'QUINTANA ROO', 24:'SAN LUIS POTOSÍ', 25:'SINALOA',
           26:'SONORA', 27:'TABASCO', 28:'TAMAULIPAS', 29:'TLAXCALA',
           30:'VERACRUZ DE IGNACIO DE LA LLAVE', 31:'YUCATÁN', 32:'ZACATECAS'}

data_entidad = data_entidad.replace({'ENTIDADNACIMIENTO': estados})

rates, logrank = survival(data_entidad, 'ENTIDADNACIMIENTO', 'EDAD', 'SOBREVIVIOPARTO')
rates.dropna().plot(kind='line',
           title='Tasa de Mortalidad Materna por lugar de nacimiento')

In [23]:
niveles = {1:'NINGUNA', 31:'PRIMARIA', 32:'NINGUNA', 51:'SECUNDARIA', 52:'PRIMARIA', 71:'PREPARATORIA',
           72:'SECUNDARIA', 81:'LICENCIATURA', 82:'PREPARATORIA', 101:'POSGRADO', 102:'LICENCIATURA',
           111:'SECUNDARIA', 112:'PRIMARIA', 131:'PREPARATORIA', 132:'SECUNDARIA'}
           
data_escolaridad = data[((data['ESCOLARIDAD'] != 0) & (data['ESCOLARIDAD'] != 88) & (data['ESCOLARIDAD'] != 99) & (data['ENTIDADNACIMIENTO'] != 0)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999)]
data_escolaridad = data_escolaridad.replace({'ESCOLARIDAD':niveles})

In [24]:
rates, logrank = survival(data_escolaridad, 'ESCOLARIDAD', 'EDAD', 'SOBREVIVIOPARTO')
rates.dropna().plot(kind='line',
           title='Tasa de Mortalidad Materna por nivel de escolaridad')

### Prueba de hipótesis

In [25]:
data_hipotesis = data[((data['ENTIDADNACIMIENTO'] != 88) & (data['ENTIDADNACIMIENTO'] != 99) & (data['ENTIDADNACIMIENTO'] != 0)) & ((data['SOBREVIVIOPARTO'] == 1) | (data['SOBREVIVIOPARTO'] == 2)) & (data['EDAD'] != 999) & ((data['SECONSIDERAINDIGENA'] == 1) | (data['SECONSIDERAINDIGENA'] == 2))]
data_hipotesis['SOBREVIVIOPARTO'].replace(1, 0, inplace=True)
data_hipotesis['SOBREVIVIOPARTO'].replace(2, 1, inplace=True)
data_hipotesis['SECONSIDERAINDIGENA'].replace(2, 'No', inplace=True)
data_hipotesis['SECONSIDERAINDIGENA'].replace(1, 'Si', inplace=True)
data_hipotesis['OrganizacionTerritorial'] = np.where(data_hipotesis['ENTIDADNACIMIENTO'] == 9, 'Capital', 'Interior')
data_hipotesis = data_hipotesis.replace({'ENTIDADNACIMIENTO': estados})

# pd.pivot_table(data_hipotesis,
#                values='SOBREVIVIOPARTO',
#                columns='SECONSIDERAINDIGENA',
#                index='ENTIDADNACIMIENTO',
#                aggfunc=np.mean)

In [26]:
interior = data_hipotesis[data_hipotesis['OrganizacionTerritorial'] == 'Interior']['SOBREVIVIOPARTO']
capital = data_hipotesis[data_hipotesis['OrganizacionTerritorial'] == 'Capital']['SOBREVIVIOPARTO']
nac_chiapas = data_hipotesis[data_hipotesis['ENTIDADNACIMIENTO'] == 'CHIAPAS']['SOBREVIVIOPARTO']
nac_edomex = data_hipotesis[data_hipotesis['ENTIDADNACIMIENTO'] == 'MÉXICO']['SOBREVIVIOPARTO']

In [27]:
len(nac_edomex)

169636

In [28]:
nacimientos = data_hipotesis.SOBREVIVIOPARTO
mu = nacimientos.mean()

In [35]:
mu

8.248300762295087e-05

In [29]:
len(nacimientos)

1709443

In [30]:
ztest(interior, value=mu)

(0.015710310396077575, 0.9874655015063141)

In [31]:
ztest(capital, value=mu)

(-0.05686151804327854, 0.9546555088434752)

In [32]:
ztest(nac_chiapas, value=mu)

(0.899118345741701, 0.36858962721789745)

In [33]:
ztest(nac_edomex, value=mu)

(0.7295603497545603, 0.46565896605448387)

### Gráficos

In [34]:
data_hipotesis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1709443 entries, 0 to 1747846
Data columns (total 65 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   NACIOEXTRANJERO             int64  
 1   ENTIDADNACIMIENTO           object 
 2   MUNICIPIONACIMIENTO         int64  
 3   EDAD                        int64  
 4   SECONSIDERAINDIGENA         object 
 5   HABLALENGUAINDIGENA         int64  
 6   FECHANACIMIENTOMADRE        object 
 7   ESTADOCONYUGAL              int64  
 8   RESIDEEXTRANJERO            int64  
 9   ENTIDADRESIDENCIA           int64  
 10  MUNICIPIORESIDENCIA         int64  
 11  LOCALIDADRESIDENCIA         int64  
 12  NUMEROEMBARAZOS             int64  
 13  HIJOSNACIDOSMUERTOS         int64  
 14  HIJOSNACIDOSVIVOS           int64  
 15  HIJOSSOBREVIVIENTES         int64  
 16  CONDICIONHIJOANTERIOR       int64  
 17  VIVEHIJOANTERIOR            int64  
 18  ORDENNACIMIENTO             int64  
 19  ATENCIONPRENATAL     